# Benchmark Code for Convolutional Neural Networks  
Accuracy and running time of Convolutional Neural Networks with Max-Pooling layers will be benchmarked here.  

## The Data  
CIFAR-10 dataset is used and is downloaded using the Keras library.  

In [4]:
import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [2]:
(X_train, Y_train), (X_test, Y_test) = utils.load_data()

In [3]:
print X_train.shape
print X_test.shape
print Y_train.shape
print Y_test.shape

(50000, 3, 32, 32)
(10000, 3, 32, 32)
(50000, 10)
(10000, 10)


## The Model  
We use a 'typical' CNN with Max-Pooling layers to predict classes in CIFAR-10 dataset.  

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

Optionally, install [Hualos](https://github.com/fchollet/hualos) and use it to visualize the loss function live.  

In [6]:
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

### Define the model

In [23]:
pool_size = (3, 3)

model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(3,32,32)))  # Layer 1
model.add(Convolution2D(32, 3, 3, activation='relu'))  # Layer 2
model.add(MaxPooling2D(pool_size=pool_size))  # Layer 3
model.add(Dropout(0.25))  # Layer 4

model.add(Convolution2D(64, 3, 3, activation='relu'))  # Layer 5
model.add(Convolution2D(64, 3, 3, activation='relu'))  # Layer 6
model.add(MaxPooling2D(pool_size=pool_size))  # Layer 7
model.add(Dropout(0.25))  # Layer 8

model.add(Flatten())  # Layer 9

model.add(Dense(512, activation='relu'))  # Layer 10
model.add(Dropout(0.5))  # Layer 11
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

Or load saved model

In [7]:
model = utils.load_model('model-big-50')

### Train  the model

In [24]:
model.fit(X_train, Y_train, nb_epoch=50, batch_size=32, verbose=1, show_accuracy=True, callbacks=[remote])

Epoch 1/50
50000/50000 [==============================] - 35s - loss: 1.8570 - acc: 0.2962    
Epoch 2/50
50000/50000 [==============================] - 35s - loss: 1.5327 - acc: 0.4452    
Epoch 3/50
50000/50000 [==============================] - 35s - loss: 1.4120 - acc: 0.4961    
Epoch 4/50
50000/50000 [==============================] - 35s - loss: 1.3381 - acc: 0.5280    
Epoch 5/50
50000/50000 [==============================] - 35s - loss: 1.2864 - acc: 0.5490    
Epoch 6/50
50000/50000 [==============================] - 36s - loss: 1.2430 - acc: 0.5667    
Epoch 7/50
50000/50000 [==============================] - 36s - loss: 1.2169 - acc: 0.5775    
Epoch 8/50
50000/50000 [==============================] - 36s - loss: 1.1852 - acc: 0.5854    
Epoch 9/50
50000/50000 [==============================] - 35s - loss: 1.1713 - acc: 0.5940    
Epoch 10/50
50000/50000 [==============================] - 36s - loss: 1.1516 - acc: 0.6028    
Epoch 11/50
50000/50000 [========================

### Test the model

In [8]:
loss, accuracy = model.evaluate(X_test, Y_test, show_accuracy=True)
print "Loss:", loss
print "Accuracy:", accuracy

10000/10000 [==============================] - 2s     
Loss: 0.756164782715
Accuracy: 0.7486


### Save the model

In [26]:
utils.save_model(model, 'model-3x3-3-50')

True